# TP2: Modelado sobre dataset Fashion MNIST

#### Conversión a csv

Fuente: https://pjreddie.com/projects/mnist-in-csv/

In [5]:
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [3]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("./data/fashionmnist/train-images-idx3-ubyte", "./data/fashionmnist/train-labels-idx1-ubyte",
        "./data/fashionmnist/mnist_train.csv", 60000)
convert("./data/fashionmnist/t10k-images-idx3-ubyte", "./data/fashionmnist/t10k-labels-idx1-ubyte",
        "./data/fashionmnist/mnist_test.csv", 10000)


In [7]:
train_df = pd.read_csv('./data/fashionmnist/mnist_train.csv',sep=',')
test_df = pd.read_csv('./data/fashionmnist/mnist_test.csv', sep = ',')

In [8]:
train_df.shape

(59999, 785)

In [9]:
test_df.shape

(9999, 785)

In [1]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels


In [ ]:
X_train, y_train = load_mnist('data/fashion', kind='train')
X_test, y_test = load_mnist('data/fashion', kind='t10k')

In [4]:
from pathlib import Path
Path("./data").mkdir(parents=True, exist_ok=True)

import requests
url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz"


from tqdm import tqdm 
r = requests.get(url, stream=True)
with open("data/file.gz", 'wb') as f:
    for chunk in tqdm(r.raw.stream(1024, decode_content=False)):
        if chunk:
            f.write(chunk)

4319it [00:03, 1171.29it/s]
